In [ ]:
import sys
sys.path.append('../src')

In [ ]:
from importlib import reload
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline

In [ ]:
import gta.vision

In [ ]:
recorder = gta.vision.VisionRecorder()
recorder.start()
time.sleep(2)
recorder.stop()

In [ ]:
recorder.results.shape

In [ ]:
print('Mean time between images is %g sec.' % np.mean(np.diff(recorder.times)))

In [ ]:
fig, ax = plt.subplots()
i = -1
ax.imshow(recorder.results[i])
times = recorder.times
ax.set_title('$t=%g$ sec' % (times[i] - times[0],));